In [1]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langgraph.graph import StateGraph, END

In [2]:
df = pd.read_csv("..\data\processed\Articulos_LLM4.csv")

In [5]:
df.head()

,seccion,titulo,url,contenido,fecha_inicio,fecha_fin,fechas,Precios,ciudad,fechas_contexto
0,ocio,Motocrossity Oasiz Madrid para niños: adrenali...,https://quehacerconlosninos.es/motocrossity-oa...,Llega a Madrid una propuesta diferente que pro...,NaN,NaN,"(None, None)",17€,Madrid,NaN
1,ocio,Las mejores verbenas de San Isidro 2025: músic...,https://quehacerconlosninos.es/verbenas-san-is...,"Madrid se viste de mantón, clavel y chulapos p...",15 de mayo,15 de mayo,"('15 de mayo', '15 de mayo')",No especificado,Madrid,"{'fecha_normalizada': '2025-05-15', 'dia_seman..."
2,ocio,"San Isidro 2025 en Madrid: tradiciones, verben...",https://quehacerconlosninos.es/san-isidro-2025...,San Isidro 2025 en Madridya tiene fecha: del 3...,15 de mayo,15 de mayo,"('15 de mayo', '15 de mayo')",No especificado,Madrid,"{'fecha_normalizada': '2025-05-15', 'dia_seman..."
3,ocio,Qué hacer en Madrid en mayo con niños: planes ...,https://quehacerconlosninos.es/que-hacer-en-ma...,Mayo es uno de los mejores meses para disfruta...,24 de mayo,30 de mayo,"('24 de mayo', '30 de mayo')",No especificado,Madrid,"{'fecha_normalizada': '2025-05-24', 'dia_seman..."
4,ocio,"Feria del Barro de Aranjuez 2025: talleres, co...",https://quehacerconlosninos.es/feria_del_barro...,Laferia del Barro de Aranjuezregresa este fin ...,domingo 27 de abril,domingo 27 de abril,"('domingo 27 de abril', 'domingo 27 de abril')",No especificado,Madrid,"{'fecha_normalizada': '2025-04-27', 'dia_seman..."


In [3]:
docs = [
    Document(
        page_content=row["contenido"],
        metadata={
            "titulo": row["titulo"],
            "url": row["url"],
            "precio": row.get("Precios", ""),
            "fecha_inicio": row.get("fecha_inicio", ""),
            "fecha_fin": row.get("fecha_fin", ""),
            "seccion": row["seccion"],
            "ciudad": row["ciudad"]
        }
    )
    for _, row in df.iterrows()
]


In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)

# Crear los Document chunked
chunked_docs = []

for _, row in df.iterrows():
    texto = row["contenido"]
    if pd.isnull(texto):
        continue
    chunks = splitter.split_text(texto)
    for i, chunk in enumerate(chunks):
        chunked_docs.append(Document(
            page_content=chunk,
            metadata={
            "titulo": row["titulo"],
            "url": row["url"],
            "precio": row.get("Precios", ""),
            "fecha_inicio": row.get("fecha_inicio", ""),
            "fecha_fin": row.get("fecha_fin", ""),
            "seccion": row["seccion"],
            "ciudad": row["ciudad"]
            }
    )
        )

In [6]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Vectorstore
vectorstore = FAISS.from_documents(chunked_docs, embedding_model)

In [9]:
import re
from datetime import datetime, timedelta
from langchain.schema import Document
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from datetime import datetime,timedelta
from langchain_community.llms import Ollama

# 1. Configura tu modelo local con Ollama
llm = Ollama(model="mistral")

class AgentState(TypedDict):
    query: str
    documents: List[Document]
    generation: str


def parse_fecha(fecha_str):
    """Intenta convertir un string como '15 de mayo' o 'domingo 27 de abril' en un objeto datetime.date"""
    try:
        fecha_str = re.sub(r'^\w+\s', '', fecha_str.strip())
        return datetime.strptime(fecha_str, "%d de %B").replace(year=datetime.now().year).date()
    except Exception:
        return None


def detectar_intencion_temporal(query):
    """Detecta si el usuario está preguntando por hoy, mañana, fin de semana, etc."""
    hoy = datetime.now().date()
    query = query.lower()

    if "hoy" in query:
        return lambda fecha_ini, fecha_fin: fecha_ini <= hoy <= fecha_fin
    elif "mañana" in query:
        manana = hoy + timedelta(days=1)
        return lambda fecha_ini, fecha_fin: fecha_ini <= manana <= fecha_fin
    elif "semana que viene" in query:
        prox_lunes = hoy + timedelta(days=(7 - hoy.weekday()))
        prox_domingo = prox_lunes + timedelta(days=6)
        return lambda fecha_ini, fecha_fin: fecha_ini <= prox_domingo and fecha_fin >= prox_lunes
    elif "fin de semana" in query:
        viernes = hoy + timedelta((4 - hoy.weekday()) % 7)
        domingo = viernes + timedelta(days=2)
        return lambda fecha_ini, fecha_fin: fecha_ini <= domingo and fecha_fin >= viernes
    else:
        return lambda fecha_ini, fecha_fin: True  # No aplica filtro temporal


def filtrar_por_fecha_y_seccion(docs, query):
    criterio_fecha = detectar_intencion_temporal(query)
    criterio_seccion = "ocio" if any(p in query.lower() for p in ["ocio", "plan", "evento", "actividad"]) else None

    docs_filtrados = []
    docs_sin_fecha = []

    for doc in docs:
        meta = doc.metadata
        seccion = meta.get("seccion", "").lower()
        fecha_ini_str = meta.get("fecha_inicio", "")
        fecha_fin_str = meta.get("fecha_fin", "")

        fecha_ini = parse_fecha(fecha_ini_str) if fecha_ini_str else None
        fecha_fin = parse_fecha(fecha_fin_str) if fecha_fin_str else fecha_ini

        if not fecha_ini and not fecha_fin:
            if criterio_seccion and seccion == criterio_seccion:
                docs_sin_fecha.append(doc)
            continue

        if fecha_ini and fecha_fin and criterio_fecha(fecha_ini, fecha_fin):
            if not criterio_seccion or seccion == criterio_seccion:
                docs_filtrados.append(doc)

    if len(docs_filtrados) < 5:
        docs_filtrados += docs_sin_fecha[:3]

    return docs_filtrados


# 3. Recuperación con MMR (ajustado para diversidad)
def retrieve(state):
    query = state["query"]
    docs = vectorstore.max_marginal_relevance_search(query, k=10, fetch_k=50)
    return {"documents": docs}

# 4. Generación usando contexto real
def generate(state):
    docs = filtrar_por_fecha_y_seccion(state["documents"], state["query"])
    query = state["query"]

    fecha_actual = datetime.now().strftime("%A, %d de %B de %Y")

    # Construir contexto claro y útil
    context = "\n\n".join(
        f"{doc.page_content}\nFuente: {doc.metadata.get('url', '')}"
        for doc in docs[:5]
    )

    # Prompt optimizado para Mistral:
    prompt = f"""
Fecha actual: {fecha_actual}

Responde de forma clara, breve y profesional a la pregunta del usuario utilizando solamente la información proporcionada en el contexto. 
Escribe una respuesta en forma de párrafo basada solo en el contexto anterior. Al final de la respuesta, agrega una lista de hasta 3 enlaces relevantes extraídos del contexto.
No inventes actividades, lugares ni enlaces. Si hay poca información, da una recomendación general y sugiere consultar los enlaces reales al final.

Contexto:
\"\"\"
{context}
\"\"\"

Pregunta:
{query}

Respuesta:
"""

    respuesta = llm.invoke(prompt)

    enlaces = {
        doc.metadata.get("url"): doc.metadata.get("titulo", "Actividad sin título")
        for doc in docs if doc.metadata.get("url")
    }
    enlaces = dict(list(enlaces.items())[:3])

    # Bloque de enlaces al final
    if enlaces:
        links_texto = (
            "\n\nPara más información sobre estas actividades, puedes consultar:\n" +
            "\n".join(f"- {titulo}\n  {url}" for url, titulo in enlaces.items())
        )
    else:
        links_texto = ""

    return {
        "generation": f"{respuesta.strip()}{links_texto}"
    }

graph = StateGraph(AgentState)
graph.add_node("retrieve", retrieve)
graph.add_node("generate", generate)
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "generate")
graph.add_edge("generate", END)

rag_agent = graph.compile()


In [10]:
# 6. Ejecuta una prueba
respuesta = rag_agent.invoke({"query": "¿Qué actividades hay en sevilla?"})
print(respuesta["generation"])

Durante las Fiestas de San Isidro en Madrid del 3 al 15 de mayo, no se ha proporcionado información específica sobre actividades en Sevilla. Sin embargo, en Madrid, se encuentran diversos espectáculos, verbenas y talleres destinados a niños en lugares como la Pradera y Las Vistillas durante los días del 10 al 15 de mayo. A continuación, te recomiendo consultar [esta guía completa para actividades con niños durante las Fiestas de San Isidro](https://quehacerconlosninos.es/san-isidro-2025-madrid-con-ninos/) y la [página oficial de Fiestas de San Isidro en Madrid](http://www.fiestasdeisidro.com/) para más información sobre actividades en otras ciudades como Sevilla.

Para más información sobre estas actividades, puedes consultar:
- San Isidro 2025 en Madrid: tradiciones, verbenas y planes para disfrutar con niños
  https://quehacerconlosninos.es/san-isidro-2025-madrid-con-ninos/
